In [21]:
import pandas as pd

In [22]:
# inname1 = 'LabelsWithExtraInfo_adni3_train.csv'
# inname2 = 'LabelsWithExtraInfo_adni3_test.csv'
# outname = 'Adni3_trainLabels_extensive2.txt'
# CreateLabelsFrame1 = pd.read_csv(inname1)
# CreateLabelsFrame1 = CreateLabelsFrame1.drop(['dicomPath','info','xmlName'],axis=1)
# CreateLabelsFrame2 = pd.read_csv(inname2)
# CreateLabelsFrame2 = CreateLabelsFrame2.drop(['dicomPath','info','xmlName'],axis=1)
# CreateLabelsFrame = pd.concat([CreateLabelsFrame1, CreateLabelsFrame2], axis=0)

In [23]:
inname = 'LabelsWithExtraInfo_adni2_test.csv'
outname = 'Adni2_testLabels_extensive3.txt'
CreateLabelsFrame = pd.read_csv(inname)
CreateLabelsFrame = CreateLabelsFrame.drop(['dicomPath','info','xmlName'],axis=1)

In [24]:
CreateLabelsFrame.head()

,ID,label,extra,description,weighting
0,/trinity/home/r098375/DDS/data/ADNI/ADNI2/test...,0,0,Axial_PD_T2_FSE,PD
1,/trinity/home/r098375/DDS/data/ADNI/ADNI2/test...,0,0,Axial_PD_T2_FSE,PD
2,/trinity/home/r098375/DDS/data/ADNI/ADNI2/test...,0,0,Axial_PD_T2_FSE,PD
3,/trinity/home/r098375/DDS/data/ADNI/ADNI2/test...,0,0,Axial_PD_T2_FSE,PD
4,/trinity/home/r098375/DDS/data/ADNI/ADNI2/test...,0,0,Axial_PD_T2_FSE,PD


In [25]:
CreateLabelsFrame['origDesc'] = CreateLabelsFrame['description']

In [26]:
CreateLabelsFrame = CreateLabelsFrame.dropna()

In [27]:
uniqueDesc = CreateLabelsFrame[CreateLabelsFrame['weighting']=='PD']['description'].unique()
print(uniqueDesc)

['Axial_PD_T2_FSE']


In [28]:
def renameDesc(desc):
    desc = desc.lower()
    if 'cwk' in desc or 'lwk' in desc or 'twk' in desc:
        return 'spinal'
    if 'localizer' in desc:
        return 'localizer'
    if 'phase' in desc:
        return 'phase'
    if 'field' in desc:
        return 'fieldmap'
    if 'rage' in desc or 'fspgr' in desc:
        return 'T1w'
    if 'cal' in desc:
        return 'calibration'
    if 'scout' in desc:
        return 'scout'
    if 'flair' in desc:
        return 'T2-FLAIR'
    if 'star' in desc:
        return 'T2star'
    if 'swi' in desc:
        return 'swi'
    if 'hippo' in desc:
        return 'hippo'
    if 'perf' in desc or 'pwi' in desc:
        return 'pwi'
    if 'asl' in desc:
        return 'asl'
    if 'dti' in desc:
        return 'dti'
    if 'cerebral_blood_flow' in desc or 'cbf' in desc:
        return 'cbf'
    if 't2' in desc:
        return 'T2w'
    if 'moco' in desc:
        return 'moco'
    if 'format' in desc:
        return 'mpr'
    if 'coronal' in desc:
        return 'T1w'


In [29]:
CreateLabelsFrame['description'] = CreateLabelsFrame['origDesc'].apply(renameDesc)

In [30]:
CreateLabelsFrame[CreateLabelsFrame['description'].isna()]

,ID,label,extra,description,weighting,origDesc


In [31]:
CreateLabelsFrame[CreateLabelsFrame['description']=='T2star']['origDesc'].unique()

array([], dtype=object)

In [32]:
CreateLabelsFrame[CreateLabelsFrame['origDesc']=='s_Axial_3TE_T2_STAR_Cumulated']

,ID,label,extra,description,weighting,origDesc


In [33]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"PD":4,
             "SWI":5, "GRE":6, "T2*":7, "DWI":8, "ADC":9, "BOLD":10,
             "FlowSensitve":11,"PWI":12,"ASL":13,
             "DTI":14,"Other":15}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    dropLabels = ['moco','mpr','hippo','spinal','unknown']
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','other','survey', 'qflow','smartbrain'] #moco is a motion corrected series
    if description in dropLabels:
        return -1
    if description in otherCategories:
        return 15
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 4
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='pd':
        return 4
    if description =='swi':
        return 5
    if description.lower() =='gre':
        return 6
    if description =='T2star':
        return 7
    if description =='dwi':
        return 8
    if description =='adc':
        return 9
    if description =='bold':
        return 10
    if description =='angio': #flowsensitive
        return 11
    if description =='pwi' or description=='cbf':
        return 12
    if 'asl' in description:
        return 13
    if description == 'dti':
        return 14
    return -1

In [34]:
CreateLabelsFrame['label'] = CreateLabelsFrame.apply(assignLabel,axis=1)

In [35]:
CreateLabelsFrame[CreateLabelsFrame['label']==-1]

,ID,label,extra,description,weighting,origDesc


In [36]:
CreateLabelsFrame = CreateLabelsFrame[CreateLabelsFrame['label']!=-1]

In [37]:
def CreateIndivudalToUnifiedNameDocument(df, outfile):
    with open(outfile, 'w') as f:
        names = df['description'].unique()
        for name in names:
            nameMatching = df[df['description']==name]['origDesc'].unique()
            nameMatching = pd.Series(nameMatching).to_string()
            f.write(f'{name}:\n{nameMatching}\n\n{"-"*25}\n')

In [38]:
outfile = 'ADNI3_NameMatching.txt'
CreateIndivudalToUnifiedNameDocument(CreateLabelsFrame, outfile)

In [39]:
CreateLabelsFrame[['ID','label','extra']].to_csv(outname,index=False, sep='\t', header=False)

In [40]:
CreateLabelsFrame['label'].value_counts()

label
4    700
2    700
Name: count, dtype: int64